In [1]:
import uuid

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

main_url = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXII#'

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


## Legislatura 64

In [3]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/00-id_data/00-id/id_legislatura64.json') as d:
    data = json.load(d)

data_inc = []
for i in data:
    if i['suplente_propietario'] == 'P':
        data_inc.append(i)
len(data_inc)  

NameError: name 'json' is not defined

In [318]:
main_url = 'http://sitl.diputados.gob.mx/LXIV_leg/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [297]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 511/511 [00:16<00:00, 30.31it/s]


In [298]:
mapping_name_party = {}
for i in names_diputados[0:251]:
    mapping_name_party[i] = 'MORENA'
    
for i in names_diputados[251: 329]:
    mapping_name_party[i] = 'PAN'
    
for i in names_diputados[329:377]:
    mapping_name_party[i] = 'PRI'
    
for i in names_diputados[377:423]:
    mapping_name_party[i] = 'PT'
    
for i in names_diputados[423:450]:
    mapping_name_party[i] = 'MC'
    
for i in names_diputados[450:474]:
    mapping_name_party[i] = 'ES'
    
for i in names_diputados[474:486]:
    mapping_name_party[i] = 'PRD'
    
for i in names_diputados[486:497]:
    mapping_name_party[i] = 'PVEM'

for i in names_diputados[497:]:
    mapping_name_party[i] = 'NA'

In [299]:
len(mapping_name_party)

500

In [303]:
def obtain_id_data(name, mapping_parties, link):
    suplente_propietario = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2)').text)
    
    if any(re.findall('ficha curricular', suplente_propietario)):
        dict_id = {}
        dict_id['legislatura'] = 64
        dict_id['id_legislador'] = str(uuid.uuid1())
        dict_id['nombre_completo'] = re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', suplente_propietario))
        dict_id['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_id['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_id['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_id['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_id['suplente_propietario'] = 'P'
        dict_id['name_suplente'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2)').text)
        dict_id['partido'] = mapping_parties[name]
        dict_id['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 64
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_sup['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_id, dict_sup])
    else:
        dict_id = {}
        dict_id['legislatura'] = 64
        dict_id['id_legislador'] = str(uuid.uuid1())
        dict_id['nombre_completo'] = name
        dict_id['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_id['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_id['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_id['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_id['suplente_propietario'] = 'P'
        dict_id['name_suplente'] = suplente_propietario
        dict_id['partido'] = mapping_parties[name]
        dict_id['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 64
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = suplente_propietario
        dict_sup['tipo_legislador'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        dict_sup['entidad'] = unidecode.unidecode(driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1)').text)
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(3)').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > div > table.cajasombra > tbody > tr > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(5)').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_id, dict_sup])

In [301]:
def obtain_id_data63(name, mapping_parties, link):
    suplente_propietario = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2) > strong > font').text
    if any(re.findall('ficha curricular', suplente_propietario)):
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', suplente_propietario))
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = name
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_prop, dict_sup])
    else:
    
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = name
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = suplente_propietario
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = suplente_propietario
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        
        return([dict_prop, dict_sup])

## ID Data

In [319]:
data_id64 = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    dicts = obtain_id_data(y, mapping_name_party, x)
    for j in dicts:
        data_id64.append(j)

100%|██████████| 500/500 [09:48<00:00,  1.18s/it]


In [320]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/legislatura64.json', 'w') as json_file:
    json.dump(data_id64, json_file)

In [323]:
names = []
for i in data_id64:
    if i['nombre_completo'].lower() not in names:
        names.append(i['nombre_completo'].lower())

In [324]:
names

['aguilar castillo heriberto marcelo',
 'ramon angel valdez mendoza',
 'aguilar linda bonifacio',
 'feliciano romero martinez',
 'aguilar molina leticia arlett',
 'liliana garcia albores',
 'aguilera brenes sebastián',
 'jorge ariff marave sanchez',
 'aguilera chairez maribel',
 'martha alicia arreola martinez',
 'alfonso ramirez cuellar',
 'aguirre valenciana javier uriel',
 'alavez ruiz aleida',
 'maria concepcion franco rodriguez',
 'alegre salazar luis javier',
 'wilbert alberto batun chulim',
 'alejandro candelaria maximino',
 'luis alfredo ortiz pantoja',
 'alfaro morales maría isabel',
 'ma. guadalupe loredo gonzales',
 'almazán burgos karla yuritzi',
 'maria imelda agustina yescas ruiz',
 'alvarado moreno maría guillermina',
 'rebeca lidia garcia vallejo',
 'ambrocio gachuz josé guadalupe',
 'oscar mejia garcia',
 'andazola gómez socorro irma',
 'carlota lara guzman',
 'andrade zavala marco antonio',
 'daniel castillo de la cruz',
 'angeles mendoza julio césar',
 'luis eduardo 

In [178]:
mapping_id_cand = {}
for i in data_id:
    mapping_id_cand[i['nombre_completo']] = i['id_legislador']

In [136]:
pd.DataFrame(data_id).to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/id_data.csv', encoding = 'utf-8')

In [141]:
df_mapping_id_cand = pd.DataFrame()
df_mapping_id_cand['name'] = list(mapping_id_cand.keys())
df_mapping_id_cand['id'] = list(mapping_id_cand.values())

In [144]:
df_mapping_id_cand.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/mapping_id_incumbent.csv', encoding = 'utf-8')

In [37]:
import json

In [179]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/diputados/id_data/legislatura64.json', 'w') as json_file:
    json.dump(data_id, json_file)

## Legislatura 63

In [248]:
main_url = 'http://sitllxiii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [11]:
def obtain_id_data63(name, mapping_parties, link):
    suplente_propietario = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(6) > td:nth-child(2) > strong > font').text
    if any(re.findall('ficha curricular', suplente_propietario)):
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = re.sub('[ ]+$', '', re.sub('[(]ficha curricular[)]', '', suplente_propietario))
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = name
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_prop, dict_sup])
    else:
    
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = name
        dict_prop['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_prop['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_prop['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_prop['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = suplente_propietario
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = suplente_propietario
        dict_sup['tipo_legislador'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong > font').text
        dict_sup['entidad'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(1) > font').text
        dict_sup['distrito'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(4) > font').text
        dict_sup['curul'] = driver.find_element_by_css_selector('body > table:nth-child(1) > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong:nth-child(7) > font').text
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        
        return([dict_prop, dict_sup])

In [249]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 511/511 [00:12<00:00, 40.26it/s]


In [250]:
mapping_name_party = {}
for i in names_diputados[0:202]:
    mapping_name_party[i] = 'pri'
    
for i in names_diputados[202: 309]:
    mapping_name_party[i] = 'pan'
    
for i in names_diputados[309:360]:
    mapping_name_party[i] = 'prd'
    
for i in names_diputados[360:410]:
    mapping_name_party[i] = 'morena'
    
for i in names_diputados[410:448]:
    mapping_name_party[i] = 'partido verde ecologista'
    
for i in names_diputados[448:469]:
    mapping_name_party[i] = 'movimiento ciudadano'
    
for i in names_diputados[469:472]:
    mapping_name_party[i] = 'alianza'
    
for i in names_diputados[472:483]:
    mapping_name_party[i] = 'encuentro social'

for i in names_diputados[483:]:
    mapping_name_party[i] = 'sin partido'

In [251]:
len(mapping_name_party)

500

## Legislatura 62

In [3]:
main_url = 'http://sitllxii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [13]:
driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table:nth-child(2) > tbody > tr:nth-child(2) > td:nth-child(2)').text

'Representación proporcional'

In [28]:
def obtain_id_data62(name, mapping_parties, link):
    suplente_propietario = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table:nth-child(2) > tbody > tr:nth-child(7) > td > span').text
    
    tipo_inc = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table:nth-child(2) > tbody > tr:nth-child(2) > td:nth-child(2)').text)
    estado = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table:nth-child(2) > tbody > tr:nth-child(3) > td:nth-child(2)').text)
    distrito = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table:nth-child(2) > tbody > tr:nth-child(4) > td:nth-child(2)').text)
    curul = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table:nth-child(2) > tbody > tr:nth-child(6) > td:nth-child(2)').text)
    
    
    if any(re.findall('Suplente de:', suplente_propietario)):
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = re.sub('[ ]+$', '', re.sub('Suplente de[:]', '', suplente_propietario))
        dict_prop['tipo_legislador'] = tipo_inc
        dict_prop['entidad'] = estado
        dict_prop['distrito'] = distrito
        dict_prop['curul'] = curul
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = name
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = tipo_inc
        dict_sup['entidad'] = estado
        dict_sup['distrito'] = distrito
        dict_sup['curul'] = curul
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_prop, dict_sup])
    else:
    
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = name
        dict_prop['tipo_legislador'] = tipo_inc
        dict_prop['entidad'] = estado
        dict_prop['distrito'] = distrito
        dict_prop['curul'] = curul
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = re.sub('[ ]+$', '', re.sub('Suplente de[:]', '', suplente_propietario))
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = re.sub('[ ]+$', '', re.sub('Suplente de[:]', '', suplente_propietario))
        dict_sup['tipo_legislador'] = tipo_inc
        dict_sup['entidad'] = estado
        dict_sup['distrito'] = distrito
        dict_sup['curul'] = curul
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        
        return([dict_prop, dict_sup])

In [4]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 512/512 [00:08<00:00, 63.38it/s]


In [6]:
len(names_diputados)

500

In [7]:
mapping_name_party = {}
for i in names_diputados[0:214]:
    mapping_name_party[i] = 'pri'
    
for i in names_diputados[214: 327]:
    mapping_name_party[i] = 'pan'
    
for i in names_diputados[327:426]:
    mapping_name_party[i] = 'prd'
    
for i in names_diputados[426:453]:
    mapping_name_party[i] = 'pve'
    
for i in names_diputados[453:465]:
    mapping_name_party[i] = 'mc'
    
for i in names_diputados[465:476]:
    mapping_name_party[i] = 'pt'
    
for i in names_diputados[476:486]:
    mapping_name_party[i] = 'alianza'
    
for i in names_diputados[486:498]:
    mapping_name_party[i] = 'morenal'

for i in names_diputados[498:]:
    mapping_name_party[i] = 'sin partido'

In [9]:
len(mapping_name_party)

500

In [20]:
import unidecode

In [29]:
data_id62 = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    dicts = obtain_id_data62(y, mapping_name_party, x)
    for i in  dicts:
        data_id62.append(i)

100%|██████████| 500/500 [02:35<00:00,  3.21it/s]


In [31]:
re.sub('Suplente: ', '', 'Suplente: Luis Fernando Mancilla Fuentes')

'Luis Fernando Mancilla Fuentes'

In [34]:
data_id62_clean = []
for i in data_id62:
    i['nombre_completo'] = re.sub('Suplente: ', '', i['nombre_completo'])
    i['name_suplente'] = re.sub('Suplente: ', '', i['name_suplente'])
    data_id62_clean.append(i)

In [38]:
len(data_id62_clean)

1000

In [39]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/00-id_data/id_legislatura62.json', 'w') as json_file:
    json.dump(data_id62_clean, json_file)

## Legislatura 61

In [40]:
main_url = 'http://sitllxi.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [41]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 512/512 [00:08<00:00, 57.74it/s]


In [42]:
len(names_diputados)

500

In [55]:
mapping_name_party = {}
for i in names_diputados[0:242]:
    mapping_name_party[i] = 'pri'
    
for i in names_diputados[242: 384]:
    mapping_name_party[i] = 'pan'
    
for i in names_diputados[384:447]:
    mapping_name_party[i] = 'prd'
    
for i in names_diputados[447:469]:
    mapping_name_party[i] = 'pve'
    
for i in names_diputados[469:483]:
    mapping_name_party[i] = 'pt'
    
for i in names_diputados[483:491]:
    mapping_name_party[i] = 'alianza'
    
for i in names_diputados[491:497]:
    mapping_name_party[i] = 'mc'
    
for i in names_diputados[497:]:
    mapping_name_party[i] = 'sin partido'

In [56]:
len(mapping_name_party)

500

In [57]:
import unidecode

In [58]:
data_id62 = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    dicts = obtain_id_data62(y, mapping_name_party, x)
    for i in  dicts:
        data_id62.append(i)

100%|██████████| 500/500 [02:52<00:00,  2.91it/s]


In [59]:
data_id62_clean = []
for i in data_id62:
    i['nombre_completo'] = re.sub('Suplente: ', '', i['nombre_completo'])
    i['name_suplente'] = re.sub('Suplente: ', '', i['name_suplente'])
    data_id62_clean.append(i)

len(data_id62_clean)

1000

In [60]:
data_id62_clean

[{'legislatura': 63,
  'id_legislador': 'e9957eae-fe1d-11ea-95ca-acde48001122',
  'nombre_completo': 'Aguero Tovar José Manuel',
  'tipo_legislador': 'Mayoria Relativa',
  'entidad': 'Morelos',
  'distrito': '2',
  'curul': 'L-445',
  'suplente_propietario': 'P',
  'name_suplente': 'Edith Flores Rodríguez',
  'partido': 'pri',
  'link_curricula': 'http://sitllxi.diputados.gob.mx/curricula.php?dipt=179'},
 {'legislatura': 63,
  'id_legislador': 'e9957ff8-fe1d-11ea-95ca-acde48001122',
  'nombre_completo': 'Edith Flores Rodríguez',
  'tipo_legislador': 'Mayoria Relativa',
  'entidad': 'Morelos',
  'distrito': '2',
  'curul': 'L-445',
  'suplente_propietario': 'S',
  'name_suplente': '',
  'partido': 'pri',
  'link_curricula': 'http://sitllxi.diputados.gob.mx/curricula.php?dipt=179'},
 {'legislatura': 63,
  'id_legislador': 'e9d6a474-fe1d-11ea-95ca-acde48001122',
  'nombre_completo': 'Aguilar González José Óscar',
  'tipo_legislador': 'Mayoria Relativa',
  'entidad': 'Puebla',
  'distrito'

In [61]:

with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/00-id_data/id_legislatura61.json', 'w') as json_file:
    json.dump(data_id62_clean, json_file)

## Legislatura 60

In [62]:
main_url = 'http://sitllx.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'
driver.get(main_url)

In [63]:
links_diputados = []
names_diputados = []
for i in tqdm(driver.find_elements_by_css_selector('a')):
    link = i.get_attribute('href')
    if any(re.findall('curricula', link)):
        links_diputados.append(link)
        names_diputados.append(re.sub('^[0-9]+ ', '', i.text))
        
    else:
        continue

100%|██████████| 512/512 [00:08<00:00, 58.83it/s]


In [83]:
len(names_diputados)

500

In [77]:
mapping_name_party = {}
for i in names_diputados[0:206]:
    mapping_name_party[i] = 'pan'
    
for i in names_diputados[206: 331]:
    mapping_name_party[i] = 'prd'
    
for i in names_diputados[331:437]:
    mapping_name_party[i] = 'pri'
    
for i in names_diputados[437:455]:
    mapping_name_party[i] = 'mc'
    
for i in names_diputados[455:472]:
    mapping_name_party[i] = 'pve'
    
for i in names_diputados[472:483]:
    mapping_name_party[i] = 'pt'
    
for i in names_diputados[483:492]:
    mapping_name_party[i] = 'alianza'
    
for i in names_diputados[492:497]:
    mapping_name_party[i] = 'psd'
for i in names_diputados[497:]:
    mapping_name_party[i] = 'sin partido'

In [81]:
len(mapping_name_party)

500

In [85]:
import unidecode

In [87]:
def obtain_id_data60(name, mapping_parties, link):
    suplente_propietario = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(8) > td > span').text
    
    tipo_inc = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(2)').text)
    estado = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(4) > td:nth-child(2)').text)
    distrito = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(5) > td:nth-child(2)').text)
    curul = unidecode.unidecode(driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(7) > td:nth-child(2)').text)
    
    
    if any(re.findall('Suplente de:', suplente_propietario)):
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = re.sub('[ ]+$', '', re.sub('Suplente de[:]', '', suplente_propietario))
        dict_prop['tipo_legislador'] = tipo_inc
        dict_prop['entidad'] = estado
        dict_prop['distrito'] = distrito
        dict_prop['curul'] = curul
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = name
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = name
        dict_sup['tipo_legislador'] = tipo_inc
        dict_sup['entidad'] = estado
        dict_sup['distrito'] = distrito
        dict_sup['curul'] = curul
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        return([dict_prop, dict_sup])
    else:
    
        dict_prop = {}
        dict_prop['legislatura'] = 63
        dict_prop['id_legislador'] = str(uuid.uuid1())
        dict_prop['nombre_completo'] = name
        dict_prop['tipo_legislador'] = tipo_inc
        dict_prop['entidad'] = estado
        dict_prop['distrito'] = distrito
        dict_prop['curul'] = curul
        dict_prop['suplente_propietario'] = 'P'
        dict_prop['name_suplente'] = re.sub('[ ]+$', '', re.sub('Suplente de[:]', '', suplente_propietario))
        dict_prop['partido'] = mapping_parties[name]
        dict_prop['link_curricula'] = link
        
        
        dict_sup = {}
        dict_sup['legislatura'] = 63
        dict_sup['id_legislador'] = str(uuid.uuid1())
        dict_sup['nombre_completo'] = re.sub('[ ]+$', '', re.sub('Suplente de[:]', '', suplente_propietario))
        dict_sup['tipo_legislador'] = tipo_inc
        dict_sup['entidad'] = estado
        dict_sup['distrito'] = distrito
        dict_sup['curul'] = curul
        dict_sup['suplente_propietario'] = 'S'
        dict_sup['name_suplente'] = ''
        dict_sup['partido'] = mapping_parties[name]
        dict_sup['link_curricula'] = link
        
        return([dict_prop, dict_sup])

In [88]:
data_id62 = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    dicts = obtain_id_data60(y, mapping_name_party, x)
    for i in  dicts:
        data_id62.append(i)

100%|██████████| 500/500 [03:07<00:00,  2.67it/s]


In [91]:
data_id62_clean = []
for i in data_id62:
    i['legislatura'] = 60
    i['nombre_completo'] = re.sub('Suplente: ', '', i['nombre_completo'])
    i['name_suplente'] = re.sub('Suplente: ', '', i['name_suplente'])
    data_id62_clean.append(i)

len(data_id62_clean)

1000

In [93]:
data_id62_clean

[{'legislatura': 60,
  'id_legislador': 'e7f3daa8-fe1f-11ea-95ca-acde48001122',
  'nombre_completo': 'Acosta Dávila Constantino',
  'tipo_legislador': 'Mayoria Relativa',
  'entidad': 'Mexico',
  'distrito': '4',
  'curul': 'K-374',
  'suplente_propietario': 'P',
  'name_suplente': 'Erika Galván Rivas',
  'partido': 'pan',
  'link_curricula': 'http://sitllx.diputados.gob.mx/curricula.php?dipt=129'},
 {'legislatura': 60,
  'id_legislador': 'e7f3dbde-fe1f-11ea-95ca-acde48001122',
  'nombre_completo': 'Erika Galván Rivas',
  'tipo_legislador': 'Mayoria Relativa',
  'entidad': 'Mexico',
  'distrito': '4',
  'curul': 'K-374',
  'suplente_propietario': 'S',
  'name_suplente': '',
  'partido': 'pan',
  'link_curricula': 'http://sitllx.diputados.gob.mx/curricula.php?dipt=129'},
 {'legislatura': 60,
  'id_legislador': 'e82a705e-fe1f-11ea-95ca-acde48001122',
  'nombre_completo': ' Efraín Arizmendi Uribe',
  'tipo_legislador': 'Representacion proporcional',
  'entidad': 'Guerrero',
  'distrito': 

In [113]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/00-id_data/id_legislatura60.json', 'w') as json_file:
    json.dump(data_id62_clean, json_file)

In [118]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/00-id_data/id_legislatura64.json') as d:
    data = json.load(d)

In [119]:
data[0]

{'legislatura': 64,
 'id_legislador': 'cc9474dc-fd0e-11ea-83d8-acde48001122',
 'nombre_completo': 'Aguilar Castillo Heriberto Marcelo',
 'tipo_legislador': 'Mayoria Relativa',
 'entidad': 'Sonora',
 'distrito': '4',
 'curul': 'K-405',
 'suplente_propietario': 'P',
 'name_suplente': 'Ramon Angel Valdez Mendoza',
 'partido': 'MORENA',
 'link_curricula': 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=250'}

## Comisiones

In [112]:
all_comisiones = []
for x, y in tqdm(zip(links_diputados, names_diputados), total = len(names_diputados)):
    driver.get(x)
    id_inc = mapping_id_cand[y]
    
    comisiones = driver.find_element_by_css_selector('body > div > table:nth-child(7) > tbody > tr > td > table:nth-child(1) > tbody > tr:nth-child(2) > td').text
    for i in re.sub('ORDINARIA', '', comisiones.split('\n')[0]).split('|'):
        if any(i):
            temp = {}
            temp['nombre_completo'] = y
            temp['id_legislador'] = id_inc
            temp['comision'] = re.sub('[ ]+$', '', re.sub('^[ ]+', '', i))
            all_comisiones.append(temp)
        else:
            continue

100%|██████████| 500/500 [10:39<00:00,  1.28s/it]


In [120]:
def dummy_remove(x):
    if any(re.findall('GRUPO DE AMISTAD', x)):
        return(1)
    else:
        return(0)
df = pd.DataFrame(all_comisiones)
df['dummy_remove'] = df.apply(lambda x: dummy_remove(x['comision']), axis = 1)

In [123]:
df_comisiones = df[df['dummy_remove'] == 0]
df_comisiones = df_comisiones[['nombre_completo', 'id_legislador', 'comision']]

In [125]:
df_comisiones